In [39]:
%%bash
pip install TextBlob
pip install python-slugify

  Running setup.py bdist_wheel for python-slugify
  Stored in directory: /Users/mr/Library/Caches/pip/wheels/eb/55/ad/1405852725145e4bc088c4a387a1af4bb9bf455b77510998f5
  Running setup.py bdist_wheel for Unidecode
  Stored in directory: /Users/mr/Library/Caches/pip/wheels/a8/85/b6/71cbccded974d5a8e15cbe7d22066d87f34329d98c7ed5d786
Successfully built python-slugify Unidecode


In [1]:
import pandas as pd
import numpy as np
import textblob as tb
import nltk
from nltk.corpus import stopwords as sw
from nltk.stem import *
import urllib2
import slugify

In [2]:
dropbox = "/Users/mr/Dropbox/moviemeta/"
movieplots = pd.read_csv(dropbox + 'movieplots.csv')
#this is a work around because CSV conversion has converted list into string
from ast import literal_eval
movieplots['plots'] = movieplots['plots'].apply(literal_eval)

In [3]:
def clean(row):
    try:
        row['year'] = int(row['year'])
        return row
    except:
        row['year'] = np.nan
        return row

In [4]:
movieplots = movieplots.apply(clean, axis = 1)
dftouse = movieplots[movieplots.year > 2010]

In [5]:
print movieplots.shape
print dftouse.shape

(259028, 5)
(81488, 5)


In [6]:
import findspark
findspark.init()
import pyspark
conf = (pyspark.SparkConf()
    .setMaster('local')
    .setAppName('pyspark')
    .set("spark.executor.memory", "2g"))
sc = pyspark.SparkContext(conf=conf)
from pyspark.sql import SQLContext
sqlsc=SQLContext(sc)   

In [61]:
plots_sdf = sqlsc.createDataFrame(dftouse)

In [8]:
def sentences(text):
    blob = tb.TextBlob(text)
    return blob.sentences

In [9]:
def stem(word):
    return PorterStemmer().stem(word)

In [10]:
def process(sents, dostop=True, dostem=True):
    stopwords = set(sw.words('english'))
    sents_processed  = []
    words_processed  = []
    for sent in sents:
        if dostop == True and dostem == True: 
            words = [stem(word) for word in sent.words if word not in stopwords]     
        elif dostop == True:
            words = [word for word in sent.words if word not in stopwords]
        elif dostem == True:
            words = [stem(word) for word in sent.words ]
        sents_processed.append(words)
        words_processed += words 
    return (sents_processed, words_processed)

In [33]:
title_plots = (plots_sdf[['title','plots']]
    .map(lambda x : (x[0], ' '.join(x[1])))
)
title_sents = (title_plots
    .map(lambda x : (x[0], sentences(x[1])))
    .map(lambda x : (x[0], process(x[1])))          
).collect()

pyspark.rdd.PipelinedRDD

In [34]:
wordcounts = (title_sents
    .flatMap(lambda x: (x[1][1]))
    .map(lambda x : (x, 1))
    .reduceByKey(lambda a,b: a+b)
    .zipWithIndex()
).cache()

In [35]:
id2word = wordcounts.map(lambda (x,y): (y,x[0])).collectAsMap()

In [52]:
vocab = wordcounts.map(lambda (x,y): (x[0],y)).collectAsMap()

In [59]:
from collections import defaultdict
def bagofwords(sent):
    d = defaultdict(int) 
    for word in sent:
        d[vocab[word]] += 1
    return d.items()
documents = title_sents.map(lambda x : bagofwords(x[1]))

In [60]:
documents.take(5)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 13.0 failed 1 times, most recent failure: Lost task 0.0 in stage 13.0 (TID 12, localhost): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/opt/apache-spark/libexec/python/lib/pyspark.zip/pyspark/worker.py", line 111, in main
    process()
  File "/usr/local/opt/apache-spark/libexec/python/lib/pyspark.zip/pyspark/worker.py", line 106, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/usr/local/opt/apache-spark/libexec/python/lib/pyspark.zip/pyspark/serializers.py", line 263, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/usr/local/opt/apache-spark/libexec/python/pyspark/rdd.py", line 1295, in takeUpToNumLeft
    yield next(iterator)
  File "<ipython-input-59-48247b6999ee>", line 7, in <lambda>
  File